# Building Spark applications with Scala API

Apache Spark is written in Scala. Scala (along with Python and Java) is among three languages supported by Spark, and in fact Scala functionality is typically added the first to new Spark releases.

## Scala language deep-dive

First, we need to learn how to apply basic arithmetic transformations to Scala collections. The basic Scala collection we are going to work in this example is List. Transformations are applied using lambda functions. Similarly to the way it is done in Python (lambda x: ...) in Scala we simply use x => ... notation. 


### Basic transformations: map

As you will see, the syntax is very permissive - various alternatives are allowed in Scala:

In [1]:
1+2

3

In [2]:
val xs = List(1, 2, 3, 4, 5)

In [3]:
xs.map({x: Int => x*2})

List(2, 4, 6, 8, 10)

In [4]:
xs map {x: Int => x*2}

List(2, 4, 6, 8, 10)

In [5]:
xs map {x => x*2}

List(2, 4, 6, 8, 10)

In [6]:
xs.map(_ * 2)

List(2, 4, 6, 8, 10)

### Basic actions: reduce

In [7]:
xs.reduce(_ + _)

15

In [8]:
xs reduce {(x: Int, y: Int) => x + y}

15

In [9]:
1 + 2 + 3 + 4 + 5

15

### Defining named functions

Along with lambda functions, one can define a regular functions. In Scala we can specify input and return types:


In [10]:
def f(x: Int): Int = x * 2

In [11]:
xs map f

List(2, 4, 6, 8, 10)

In [12]:
def f(x: Int) = x * 2

In [13]:
xs map f

List(2, 4, 6, 8, 10)

In [14]:
def f(x: Int) = x * 2.0

In [15]:
xs map f

List(2.0, 4.0, 6.0, 8.0, 10.0)

In [16]:
def f(x: Int) = {
 val intermediate = x * 2
 intermediate * 3
}

In [17]:
f(10)

60

In [18]:
def f(x: Int) = {
 val intermediate = x * 2
 (intermediate, intermediate * 3)
}

In [19]:
f(10)

(20,60)

### Object oriented Scala 

In [1]:
class Dog

In [2]:
class Person {
 def walk(d: Dog) {
 println("arf! arf!")
 }
}

In [3]:
val p = new Person
val d = new Dog

In [4]:
p walk d

arf! arf!


In [5]:
case class Dog()
case class Person() {
  def walk(d: Dog) {
    println("arf! arf!")
  }
}

val p = Person()
val d = Dog()

In [6]:
p walk d

arf! arf!


In [7]:
p

Person()

In [8]:
d

Dog()

In [11]:
case class Cat(color: String) {
  def meow() {
    println("Meow!")
  }
}

In [12]:
case class Person(height: Double) {
  def walk(d: Cat) {
    println("here, kittie!")
    d.meow()
  }
}


val pp = Person(12.0)
val c = Cat("blue")

In [13]:
pp walk c

here, kittie!
Meow!


In [14]:
object Elvis extends Person(33.0)

In [17]:
Elvis walk c

here, kittie!
Meow!


In [18]:
Elvis.height

33.0

## Preparing our first Scala Spark application

Let us start with an PySpark application we have prepared on one of the previous steps. Here it is:

```python
#from pyspark import SparkContext
import sys
import time

def main1(args):
    start = time.time()
    #sc = SparkContext(appName="LoadUnstructured")

    input_rdd = sc.textFile("../2_LoadingData/data/unstructured/",10)
    counts = input_rdd.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

    print "\nTaking the 10 most frequent words in the text and corresponding frequencies:"
    print counts.takeOrdered(10, key=lambda x: -x[1])
    end = time.time()
    print "Elapsed time: ", (end-start)
```

The output of the function looks something like this:

```bash

Taking the 10 most frequent words in the text and corresponding frequencies:
[(u'the', 22635), (u'of', 11167), (u'and', 11086), (u'to', 10707), (u'a', 10433), (u'I', 10183), (u'in', 7006), (u'that', 6911), (u'was', 6779), (u'his', 4955)]
Elapsed time:  3.28145909309
```

The same application can be re-written in Scala as:

In [5]:
import org.apache.spark.SparkConf
//import org.apache.spark.sql.SparkSession

object WordCount {
  def main() { 

    val t0 = System.nanoTime()
    val conf = new SparkConf().setAppName("WordCount")


    val textFile = sc.textFile("../2_LoadingData/data/unstructured/",10)
    val counts = textFile.flatMap(line => line.split(" "))
                 .map(word => (word, 1))
                 .reduceByKey(_ + _).map(x=>x.swap).sortByKey(false)
                 
    println("\nTaking the 10 most frequent words in the text and corresponding frequencies:")
    for (d <- counts.take(10)) { 
        println(d)
    }
        
    val t1 = System.nanoTime()
    println("Elapsed time: " + (t1 - t0)/1000000000.0)
    //Do not stop the SparkContext in the jupyter notebook.... sc.stop()
  }
}

In [6]:
WordCount.main()

(224428,)
(22635,the)
(11167,of)
(11086,and)
(10707,to)
(10433,a)
(10183,I)
(7006,in)
(6911,that)
(6779,was)
Elapsed time: 1.060742867


Go back to the course working area on Adroit and change to the folder for this exercise:

```bash
cd 6_ScalaApps
```

## Submitting Scala Spark application Q/A

Q: So you've written some Spark code in Scala. How do you submit it to Spark and run it?  
A: Use `sbt` or `maven` to package it into a Java jar, and submit it to Spark using `spark-submit`

Q: What's a Java jar?  
A: JAR (Java Archive) is a package file format typically used to aggregate many Java class files and associated metadata and resources (text, images, etc.) into one file to distribute application software or libraries on the Java platform.

### Highlighting Scala syntax (on liner)

To enable Scala syntax highliting in vim, execute a following bash one-liner:
```bash
mkdir -p ~/.vim/{ftdetect,indent,syntax} && for d in ftdetect indent syntax ; do wget --no-check-certificate -O ~/.vim/$d/scala.vim https://raw.githubusercontent.com/derekwyatt/vim-scala/master/syntax/scala.vim; done
```

### Packaging with `sbt`

**What is SBT?**  
SBT is a modern build tool written in/for Scala, though it is also a general purpose build tool  

**Why SBT?**
- Good dependency management
- Full Scala language support for creating tasks
- Launch REPL in project context

Create a root directory for your project and run:
```bash
mkdir -p src/{main,test}/{resources,scala}
mkdir lib project
```
within it. 

This script will automatically create the proper `sbt` directory structure, which borrows from the Java `maven` directory structure. The script will also generate a template `build.sbt` file at the top of the directory that you should fill out with the appropriate versions and dependencies for your app.

Then we can take our Scala code, and put it in the src folder (you should have it in the main folder, so just move it there):

```bash
mv WordCount.scala src/main/scala/
```

**Project Layout (Directory structure)**   

`project` – project definition files  
`project/build/` *yourproject* `.scala` – the main project definition file  
`project/build.properties` – project, sbt and scala version definitions  
`src/main` – your app code goes here, in a subdirectory indicating the code’s language (e.g. src/main/scala, src/main/java)  
`src/main/resources` – static files you want added to your jar (e.g. logging config)  
`src/test` – like src/main, but for tests  
`lib_managed` – the jar files your project depends on. Populated by sbt update  
`target` – the destination for generated stuff (e.g. generated thrift
code, class files, jars)  

#### `build.sbt`: Dependencies and versioning

Example `simple.sbt` (located in the root directory of your project) 

```scala
name := "WordCount"

version := "1.0"

scalaVersion := "2.11.7"

libraryDependencies ++= Seq(
    // Spark dependency
    "org.apache.spark" % "spark-core_2.11" % "2.1.0" % "provided"
)
```


#### Assembly.sbt to build a fat Jar

Example assembly.sbt located in the /project folder of your project:

```scala
resolvers += Resolver.url("artifactory", url("http://scalasbt.artifactoryonline.com/scalasbt/sbt-plugin-releases"))(Resolver.ivyStylePatterns)

addSbtPlugin("com.eed3si9n" % "sbt-assembly" % "0.13.0")
```

### Running (submitting a `jar` to Spark)

1. Run `sbt assembly` in your project's home directory. The output to console will tell you the name and location of the resulting jar (under `./target`) 

You should now see the Jar file generated:
```bash
[alexeys@bd scala_spark] ll target/scala-2.11/
total 6968
drwxr-xr-x. 2 alexeys cses    4096 Dec  9 10:43 classes
-rw-r--r--. 1 alexeys cses 7129172 Dec  9 10:43 WordCount-assembly-1.0.jar
```

2. In the Slurm batch script, use spark-submit as usual to submit the Spark app, but you would need to specify the --class and the path to jar from the current folder, for instance:

```bash
spark-submit --class "WordCount" --total-executor-cores target/scala-2.11/WordCount-assembly-1.0.jar
```

### Hands-on mini-exercise

Run the above steps in your working area on Adroit.

# Model selection via cross validation

An important task in ML is model selection, or using data to find the best model or parameters for a given task. **Pipelines** facilitate model selection by making it easy to tune an entire **Pipeline** at once, rather than tuning each element in the **Pipelines** separately.

Currently, **spark.ml** supports model selection using the **CrossValidator** class, which takes an Estimator, a set of ParamMaps, and an Evaluator. CrossValidator begins by splitting the dataset into a set of folds which are used as separate training and test datasets; e.g., with k=3 there is 3 folds, CrossValidator will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. CrossValidator iterates through the set of ParamMaps. For each ParamMap, it trains the given Estimator and evaluates it using the given Evaluator.

The Evaluator can be a RegressionEvaluator for regression problems, a BinaryClassificationEvaluator for binary data, or a MultiClassClassificationEvaluator for multiclass problems. The default metric used to choose the best ParamMap can be overriden by the setMetric method in each of these evaluators.

The ParamMap which produces the best evaluation metric (averaged over the k folds) is selected as the best model. CrossValidator finally fits the Estimator using the best ParamMap and the entire dataset.

The following example demonstrates using CrossValidator to select from a grid of parameters. To help construct the parameter grid, we use the ParamGridBuilder utility.

In [11]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

// Prepare training data from a list of (id, text, label) tuples.
val training = spark.createDataFrame(Seq(
  (0L, "a b c d e spark", 1.0),
  (1L, "b d", 0.0),
  (2L, "spark f g h", 1.0),
  (3L, "hadoop mapreduce", 0.0),
  (4L, "b spark who", 1.0),
  (5L, "g d a y", 0.0),
  (6L, "spark fly", 1.0),
  (7L, "was mapreduce", 0.0),
  (8L, "e spark program", 1.0),
  (9L, "a e c l", 0.0),
  (10L, "spark compile", 1.0),
  (11L, "hadoop software", 0.0)
)).toDF("id", "text", "label")

// Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
val tokenizer = new Tokenizer().setInputCol("text").setOutputCol("words")
val hashingTF = new HashingTF().setInputCol(tokenizer.getOutputCol).setOutputCol("features")
val lr = new LogisticRegression().setMaxIter(10)
val pipeline = new Pipeline().setStages(Array(tokenizer, hashingTF, lr))

// We use a ParamGridBuilder to construct a grid of parameters to search over.
// With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
// this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
val paramGrid = new ParamGridBuilder().addGrid(hashingTF.numFeatures, Array(10, 100, 1000)).addGrid(lr.regParam, Array(0.1, 0.01)).build()

// We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
// This will allow us to jointly choose parameters for all Pipeline stages.
// A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
// Note that the evaluator here is a BinaryClassificationEvaluator and its default metric
// is areaUnderROC.
val cv = new CrossValidator().setEstimator(pipeline).setEvaluator(new BinaryClassificationEvaluator).setEstimatorParamMaps(paramGrid).setNumFolds(2) // Use 3+ in practice

// Run cross-validation, and choose the best set of parameters.
val cvModel = cv.fit(training)

// Prepare test documents, which are unlabeled (id, text) tuples.
val test = spark.createDataFrame(Seq(
  (4L, "spark i j k"),
  (5L, "l m n"),
  (6L, "mapreduce spark"),
  (7L, "apache hadoop")
)).toDF("id", "text")

// Make predictions on test documents. cvModel uses the best model found (lrModel).
cvModel.transform(test).select("id", "text", "probability", "prediction").collect().foreach { 
    case Row(id: Long, text: String, prob: Vector, prediction: Double) =>
    println(s"($id, $text) --> prob=$prob, prediction=$prediction")
  }

(4, spark i j k) --> prob=[0.1256626071135714,0.8743373928864286], prediction=1.0
(5, l m n) --> prob=[0.995215441016286,0.004784558983714055], prediction=0.0
(6, mapreduce spark) --> prob=[0.3069689523262592,0.6930310476737409], prediction=1.0
(7, apache hadoop) --> prob=[0.8040279442401363,0.1959720557598636], prediction=0.0


### Hands-on mini-exercise

1) Move the ModelSelection_scala file into the folder designedtaed for Scala source files, and rename it to have a .scala extension:

```bash
mv ModelSelection\_scala src/main/scala/ModelSelection.scala
```

The reason why the file name ends with \_scala is because this way sbt is not going to try to include it in the build at the first place. A few additions are going to be required to make this build successful, which is a part of the exercise.


2) Try building the new project:

```bash
sbt assembly
```

From the compiler error message, we can see that it does not recognize any of the Spark SQL or the ML packages. We did include those in the .scala source file, but we neeed to also include them in the SBT build file.

Now open the simple.sbt and add the following lines to the dependencies:

```bash
  "org.apache.spark"  % "spark-sql_2.11" % "2.1.0" % "provided",
  "org.apache.spark"  % "spark-mllib_2.11" % "2.1.0" % "provided" 
```  

(watch out for the commas)

Now issue the sbt assembly command again.

We still see SBT complaining about SQLContext not being defined, let us look into the source file: we see that the SQLContext is not imported, nor is it defined in the main section of the application. You can accomplish both by uncommenting the lines which I have put in place for that purpose. And repeat the SBT assembly.

We should be all set now!

3) Make adjustments to the Slurm submission file if you find it necessary and submit the job to the cluster:

```bash
sbatch slurm_for_scala.cmd
```

## Spark web UI

Spark web UI provides is the main tools for debugging and profiling Spark applications. It is easily available for Scala applications launched with spark-shell with master local.

Open a terminal window and launch spark-shell like this:

```bash
spark-shell --jars answers/ScalExample-assembly-1.0.jar
```

You will see following lines as it starts up:

```bash
$ spark-shell --jars answers/ScalExample-assembly-1.0.jar 
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
16/10/26 20:24:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Spark context Web UI available at http://128.112.173.130:4040
Spark context available as 'sc' (master = local[*], app id = local-1477527869397).
Spark session available as 'spark'.
Welcome to

      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.1.0-SNAPSHOT
      /_/

```

Among other things this contains a URL and a port of the Spark web UI:

```bash
Spark context Web UI available at http://128.112.173.130:4040
```

Copy this link and paste it in a browser window. Next, run your application (this is going to be ModelSelection class, we will run the main method):

```scala
scala> val myprogram = ModelSelection
myprogram: ModelSelection.type = ModelSelection$@104a287c

scala> myprogram.main(Array(""))
```

then refresh the browser window where you navigated to Spark web UI. Among other things you can see DAG visualization with all the stages that program goes through:

<img src="./answers/webui.png">